In [2]:
 from numpy import uint64

Operation `x^y mod z`  is widely used in cryptographic calculations, unfortunately straightforward interpretation requires manipulation of big numbers, that requires a lot of computation space and time. To fight that need operation `x^y mod x` was optimized using  several procedures.

In [3]:

def mod_power(base:uint64,exp:uint64,mod:uint64)->uint64:
    # 1. obtain binary representation of a exponent
    #    used order small endian
    bits = [int(v) for v in bin(exp)[:1:-1]]

    # 2. calculate all powers of base up to length of bits
    power_tree = [base % mod, (base * base) % mod]
    for id, bit in enumerate(bits[2:]):
        power_tree.append((power_tree[-1]*power_tree[-1]) % mod)
            
    
    # 3. multiply all partial powers from power tree, if bit of exponent is == 1     
    result = 1
    for power, bit in zip(power_tree,bits):
        if bit:
            result = (result * power) % mod
    return result


In [4]:
import random
def mod_power_test():
    print("mod_power_test...")
    for i in range(1_000):
        base = random.randint(100,100_000)
        exp = random.randint(100,100_000)
        mod = random.randint(10,10_000)
        if mod_power(base,exp,mod) != ((base ** exp) % mod):
            print(f"failed for {base}, {exp}, {mod}")
            Exception("Incorrect result")
    print("ok")
mod_power_test()


mod_power_test...
ok


# [Diffie–Hellman key exchange](https://en.wikipedia.org/wiki/Diffie%E2%80%93Hellman_key_exchange)
Assures safe key exchange between two parties.

Algorithm looks as follows:

1. Alice and Bob publicly agree to use a modulus `p = 23` and base `g = 5` (which is a primitive root modulo 23).

2. Alice chooses a secret integer `a = 4`, then sends Bob `A = g*a mod p`
    
    `A = 54 mod 23 = 4`

3. Bob chooses a secret integer `b = 3`, then sends Alice `B = g*b mod p`

    `B = 53 mod 23 = 10`

4. Alice computes `s = B*a mod p`

    `s = 104 mod 23 = 18`

5. Bob computes `s = A*b mod p`

    `s = 43 mod 23 = 18`

6.Alice and Bob now share a secret (the number 18).


# Inverse modulo power 
To perform [RSA](https://en.wikipedia.org/wiki/RSA_(cryptosystem)) algorithm special operation is used to determine [modular multiplicative inverse](https://en.wikipedia.org/wiki/Modular_multiplicative_inverse) of specified value. To perform this operation [Extended Euclidean algorithm](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm) is used.
Function below implements extended euclidean algorithm to solve for d: e * d mod fi(n) = 1

In [5]:
def modular_multiplicative_inverse(a:uint64, b:uint64):
    # q = (0,0)
    r = [a,b]
    s = [1,0]
    if a < b :
        r = [b,a]
        s = [0,1]
    while True:
        
        if r[1] == 1:
            return (r[1], abs(s[1]))
                
        elif r[1] == 0:
            return None
        
        q = r[0] // r[1]
        
        print(f"q = {q}, r  = {r} s = {s}")
        
        temp_r = r[0] - (r[1] * q)
        temp_s = s[0] - (s[1] * q)
        
        if temp_s < 0:
            temp_s += b
        
        r[0] = r[1]
        s[0] = s[1]
        
        r[1] = temp_r
        s[1] = temp_s
        
        
# example (7 * x) mode 10 = 1
print(modular_multiplicative_inverse(a = 7, b = 10))


q = 1, r  = [10, 7] s = [0, 1]
q = 2, r  = [7, 3] s = [1, 9]
(1, 7)


# RSA 
Is a public-key cryptosystem that is widely used for secure data transmission, The acronym "RSA" comes from the surnames of Ron Rivest, Adi Shamir and Leonard Adleman.
In a public-key cryptosystem, the encryption key is public and distinct from the decryption key, which is kept secret (private). An RSA user creates and publishes a public key based on two large prime numbers, along with an auxiliary value. The prime numbers are kept secret. Messages can be encrypted by anyone, via the public key, but can only be decoded by someone who knows the prime numbers.

## Key Generation
1. Chose two distinct prime numbers `p`, `q`, both should be keep as a secret
2. Compute `n = pq`, `n` is a part of public key, length of `n` in bits represents key length 
3. Compute `fi(n)` where `fi(n) = loves_common_multiple(p-1 , q-1)` `fi(n)` should be kept as a secret
4. Choose an integer `e` such that `1 < e < λ(n)` and `gde(e, λ(n)) = 1`; that is, `e` and `λ(n)` are co-primes, `e` is released as part of the public key.
Usually `e` is chosen to be `65537`   
5. Determine `d` as `d ≡ e^−1 (mod λ(n))`; that is, `d` is the modular multiplicative inverse of `e` modulo `λ(n)`, in other words `(d*e) mod fi(n) = 1`, d is kept secret as the private key exponent.

## RSA does not prevent "man in the middle" attacks